In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [2]:
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_data = train_datagen.flow_from_directory('train',
                                              target_size=(128, 128),
                                              batch_size=32,
                                              class_mode='binary')

test_data = test_datagen.flow_from_directory('test',
                                            target_size=(128, 128),
                                            batch_size=32,
                                            class_mode='binary')

Found 2637 images belonging to 2 classes.
Found 581 images belonging to 2 classes.


In [3]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(128, 128, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

In [4]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [5]:
model.fit(train_data,
          steps_per_epoch=train_data.samples // 32,
          epochs=10,
          validation_data=test_data,
          validation_steps=test_data.samples // 32)

Epoch 1/10
82/82 [==============================] - 53s 638ms/step - loss: 0.6101 - accuracy: 0.6583 - val_loss: 0.5386 - val_accuracy: 0.7326
Epoch 2/10
82/82 [==============================] - 30s 370ms/step - loss: 0.5221 - accuracy: 0.7374 - val_loss: 0.4457 - val_accuracy: 0.8021
Epoch 3/10
82/82 [==============================] - 34s 409ms/step - loss: 0.4368 - accuracy: 0.7881 - val_loss: 0.4174 - val_accuracy: 0.7865
Epoch 4/10
82/82 [==============================] - 40s 488ms/step - loss: 0.4178 - accuracy: 0.7873 - val_loss: 0.4064 - val_accuracy: 0.8073
Epoch 5/10
82/82 [==============================] - 46s 564ms/step - loss: 0.4142 - accuracy: 0.8038 - val_loss: 0.3851 - val_accuracy: 0.8073
Epoch 6/10
82/82 [==============================] - 44s 536ms/step - loss: 0.4070 - accuracy: 0.8008 - val_loss: 0.3679 - val_accuracy: 0.8333
Epoch 7/10
82/82 [==============================] - 39s 467ms/step - loss: 0.4074 - accuracy: 0.8073 - val_loss: 0.3661 - val_accuracy: 0.8299

In [6]:
model.save('skin1.h5')

C:\Users\gsuma\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
from tensorflow.keras.models import load_model
model = load_model('skin1.h5')
def preprocess_image(image_path):
    from tensorflow.keras.preprocessing import image
    img = image.load_img(image_path, target_size=(128, 128))
    img = image.img_to_array(img)
    img = img / 255.0 
    img = np.expand_dims(img, axis=0) 
    return img
def predict_image(image_path):
    img = preprocess_image(image_path)
    prediction = model.predict(img)

    if prediction > 0.5:
        return 'benign'
    else:
        return 'malignant'

image_path = 'test/malignant/1.jpg'
result = predict_image(image_path)
print(f'The image is a {result}')
image_path

1/1 [==============================] - 0s 167ms/step
The image is a benign


'test/malignant/1.jpg'

In [8]:
import os
from sklearn.metrics import *
test_dir = r'test'

true_labels = []
predicted_labels = []
for class_name in ['benign', 'malignant']:
    class_dir = os.path.join(test_dir, class_name)
    for image_filename in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_filename)
        true_labels.append(class_name)

        result = predict_image(image_path)
        predicted_labels.append(result)

confusion = confusion_matrix(true_labels, predicted_labels)
accuracy = accuracy_score(true_labels, predicted_labels)
class_report = classification_report(true_labels, predicted_labels)

1/1 [==============================] - 0s 36ms/step


In [9]:
print("Confusion Matrix:\n", confusion)
print("\nAccuracy:", accuracy)
print("\nClassification Report:\n", class_report)

Confusion Matrix:
 [[ 55 226]
 [260  40]]

Accuracy: 0.16351118760757316

Classification Report:
               precision    recall  f1-score   support

      benign       0.17      0.20      0.18       281
   malignant       0.15      0.13      0.14       300

    accuracy                           0.16       581
   macro avg       0.16      0.16      0.16       581
weighted avg       0.16      0.16      0.16       581



In [10]:
loss, accuracy = model.evaluate(test_data)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

19/19 [==============================] - 2s 74ms/step - loss: 0.3658 - accuracy: 0.8365
Test loss: 0.3658
Test accuracy: 0.8365
